In [ ]:
#Daywise variance and slope
#cluser day wise
#cluster of COuntries
# day wise data structure D0 to Dn
# attributes of a timeseriess
# how we are placed compared to other countries
#lockdown impact on graphs
#day wise story time  graphs of countries
#predictions of countries
# predictions of the countries compared to actual

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing,cross_decomposition,model_selection,metrics
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
%matplotlib inline
import dateutil
from tqdm import tqdm
from sklearn import linear_model

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [ ]:
confirmedGlobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',encoding='utf-8',na_values=None)
deathGLobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoverGlobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
confirmedGlobal['NewContryCode']=confirmedGlobal['Country/Region']+confirmedGlobal['Province/State'].fillna('')
deathGLobal['NewContryCode']=deathGLobal['Country/Region']+deathGLobal['Province/State'].fillna('')
recoverGlobal['NewContryCode']=recoverGlobal['Country/Region']+recoverGlobal['Province/State'].fillna('')

In [ ]:
confirmedGlobal.head()

In [ ]:
def getStartPoint(country):
    for num,i in enumerate(confirmedGlobal[confirmedGlobal['NewContryCode']==country].values[0][4:]):
        if i > 0:
#             print (num,i)
            break
    return 4+num

def getSlope(dataSet):
    df=pd.DataFrame(dataSet)
    df=df.reset_index()
    df.columns=['x','y']
    model=linear_model.LinearRegression(fit_intercept=False).fit(df[['x']],df[['y']])
    return model.coef_[0][0]
def getDataArranged(country):
    temp={}
    

    startPoint=getStartPoint(country)
    temp['logValues']=[np.log(i) if i !=0  else 0 for i in confirmedGlobal[confirmedGlobal['NewContryCode']==country].values[0][startPoint:-1]]
    temp['actualValues']=[i for i in confirmedGlobal[confirmedGlobal['NewContryCode']==country].values[0][startPoint:-1]]
    countryData=confirmedGlobal[confirmedGlobal['NewContryCode']==country].values[0][startPoint:-1]
    datesOfInfections=[str(dateutil.parser.parse(i))[:10] for i in confirmedGlobal[confirmedGlobal['NewContryCode']==country].columns[startPoint:-1]]
    dayIndex=['D_'+str(i).zfill(3) for i in range(len(countryData))]
    temp['countryData']=countryData
    lbl=preprocessing.MinMaxScaler()
    temp['minMaxScaledval']=np.ravel(lbl.fit_transform(countryData.reshape(len(countryData),1)))
    temp['varianceVal']=np.var(temp['minMaxScaledval'])
    temp['slopeData']=getSlope(countryData)
    temp['datesOfInfections']=datesOfInfections
    temp['dayIndex']=dayIndex
    temp['lastValue']=countryData[-1]
    return temp

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import tsa

In [ ]:
countryDataDict={}
for con in tqdm(confirmedGlobal['NewContryCode']):
    try:
        countryDataDict[con]=getDataArranged(con)    
    except:
        pass

In [ ]:
tempX=countryDataDict['India']

In [ ]:
tempX.keys()

In [ ]:
def getCountryDayWiseDetails(country):
    indiCountyDaywise=[]
    tempX=countryDataDict[country]
    for i in range(len(tempX['countryData'])):
        try:
            tempDict={'variancDaywise':np.var(tempX['minMaxScaledval'][:i]),
                  'dayInfo':i,#'D_'+str(i).zfill(3),
                  'slopeVal':getSlope(tempX['countryData'][:i]),
                'infected':tempX['countryData'][i]}
        except:
            tempDict={'variancDaywise':None,
                  'dayInfo':i,#'D_'+str(i).zfill(3),
                  'slopeVal':None,
                     'infected':tempX['countryData'][i]}

        indiCountyDaywise.append(tempDict)
    tempDWiseData=pd.DataFrame(indiCountyDaywise)
    tempDWiseData['CountryName']=country
    return tempDWiseData

In [ ]:
countryDataDictDayWise=[]
for con in tqdm(countryDataDict):
    countryDataDictDayWise.append(getCountryDayWiseDetails(con))

## Slider Details

In [ ]:
allCOuntryDayWiseDataDF=pd.concat(countryDataDictDayWise)

In [ ]:
allCOuntryDayWiseDataDF.head()

In [ ]:
max(allCOuntryDayWiseDataDF['slopeVal'].fillna(0))

In [ ]:
allCOuntryDayWiseDataDF[allCOuntryDayWiseDataDF['slopeVal']>1960]

In [ ]:
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [ ]:
daySLider = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=max(allCOuntryDayWiseDataDF['dayInfo']),
    step=1.0,
    description='Day:',
    continuous_update=False
)

# textbox = widgets.Dropdown(
#     description='Country:   ',
#     value='India',
#     options=allCOuntryData.columns.tolist()
# )
container = widgets.HBox(children=[daySLider])
# from plotly.subplots import make_subplots

In [ ]:
tempDFFOrSlider=allCOuntryDayWiseDataDF[allCOuntryDayWiseDataDF['dayInfo']==59]
trace1 = go.Scatter(
    x=tempDFFOrSlider['slopeVal'],
    y=tempDFFOrSlider['variancDaywise'],
#     mode='markers',
     mode="markers+text",
    marker=dict(size=tempDFFOrSlider['infected'],sizemode='area',
        sizeref=2.*max(tempDFFOrSlider['infected'])/(40.**2),
        sizemin=4),
    showlegend=True,
    text=tempDFFOrSlider['CountryName'])

In [ ]:
g = go.FigureWidget(data=[trace1],layout=go.Layout(title=dict(text='Covid19'),barmode='overlay'))

In [ ]:
def response(change):
    vaL=daySLider.value
    tempDFFOrSlider=allCOuntryDayWiseDataDF[allCOuntryDayWiseDataDF['dayInfo']==vaL]
    x1 = tempDFFOrSlider['slopeVal'].values
    x2 = tempDFFOrSlider['variancDaywise'].values
    with g.batch_update():
        g.data[0].x = x1
        g.data[0].y = x2
        g.data[0].marker['size']= tempDFFOrSlider['infected'].values
        g.layout.barmode = 'overlay'
daySLider.observe(response, names="value")
# textbox.observe(response, names="value")

In [ ]:
widgets.VBox([container,g])

## All Country Information